## Original model 

In [34]:
var_list

[<gurobi.Var Stock0 (value 0.49999988676742535)>,
 <gurobi.Var Stock1 (value 4.5737635771337173e-07)>,
 <gurobi.Var Stock2 (value 0.4999996558562219)>]

In [36]:
covariances

[[0.04, 0.018, 0.0064], [0.018, 0.0225, 0.0084], [0.0064, 0.0084, 0.0064]]

In [1]:
from gurobipy import *
import numpy as np
import math
m = Model('portfolio')


#data impute
mean_return = [0.14, 0.11, 0.1]
stdev_return = [0.2, 0.15, 0.08]
correlations = [[1,0.6,0.4],
                [0.6,1,0.7],
                [0.4,0.7,1]]
covariances = [[0.04,0.018,0.0064],
               [0.018,0.0225,0.0084],
               [0.0064,0.0084,0.0064]]
required_mean_return = 0.12
required_total_weights = 1.0

# add variable
var_list=[m.addVar(vtype=GRB.CONTINUOUS,name="Stock"+str(i)) for i in range(3)]
m.update()
PortVar = np.dot(var_list, np.dot(covariances,np.transpose(var_list)))

# add constraints 
total_weights = sum(var_list)
mean_portfolio_return = var_list[0]*mean_return[0] + var_list[1]*mean_return[1] + var_list[2]*mean_return[2]
m.addConstr(total_weights, GRB.EQUAL, required_total_weights)
m.addConstr(mean_portfolio_return, GRB.GREATER_EQUAL, required_mean_return)
m.update()

m.setObjective(PortVar, GRB.MINIMIZE)
m.update()
m.optimize()


PortSd = math.sqrt(m.objval)
print('the portfolio standar deviation is: ',PortSd)

Academic license - for non-commercial use only
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model has 6 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-02, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros
Presolved model has 6 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.39490847e+05 -1.39490847e+05  2.50e+03 6.11e-06  1.00e+06     0s
   1   4.91553548e+03 -5.16907979e+03  1.49e+02 3.64e-07  6.17e+04     0s
   2   3.34728926e-02 -3.07903637e+02  1.50e-01 3.67e-10  1.39e+02     0s
   3   2.42930548e-02 -5.4

In [2]:
PortVar

<gurobi.QuadExpr: 0.0 + [ 0.04 Stock0 ^ 2 + 0.018 Stock0 * Stock1 + 0.0064 Stock0 * Stock2 + 0.018 Stock1 * Stock0 + 0.0225 Stock1 ^ 2 + 0.0084 Stock1 * Stock2 + 0.0064 Stock2 * Stock0 + 0.0084 Stock2 * Stock1 + 0.0064 Stock2 ^ 2 ]>

## Connect SQL

In [5]:
import mysql.connector
db = mysql.connector.connect(user='root', password = 'root', database ='nasdaq2', 
                             auth_plugin = 'mysql_native_password')
cur = db.cursor() #create an empty cursor
cur.description
cur.execute('select covariance from cov')

In [6]:
a=cur.fetchall()

In [7]:
pre_q=[i[0] for i in a]

In [8]:
covariances=[pre_q[i:i+1158] for i in range(0,len(pre_q),1158)]
    

In [9]:
cur1 = db.cursor() #create an empty cursor
cur1.description
cur1.execute('select mean from r')
b=cur1.fetchall()

In [10]:
mean_return=[i[0] for i in b]

In [11]:
cur2 = db.cursor() #create an empty cursor
cur2.description
cur2.execute('select stdev from stdev')
c=cur1.fetchall()
stdev_return=[i[0] for i in c]

In [12]:
from gurobipy import *
import numpy as np
import math
m = Model('portfolio')


#data impute
#mean_return = [0.14, 0.11, 0.1]
#stdev_return = [0.2, 0.15, 0.08]

'''
correlations = [[1,0.6,0.4],
                [0.6,1,0.7],
                [0.4,0.7,1]]
'''
'''
covariances = [[0.04,0.018,0.0064],
               [0.018,0.0225,0.0084],
               [0.0064,0.0084,0.0064]]
'''
risk = 0.05
required_total_weights = 1.0

# add variable
var_list=[m.addVar(vtype=GRB.CONTINUOUS,name="Stock"+str(i)) for i in range(1158)]
m.update()
PortVar=quicksum(var_list[i]*covariances[i][j]*var_list[j] for i in range(1158) for j in range(1158))
#PortVar = np.dot(var_list, np.dot(covariances,np.transpose(var_list)))

# add constraints 
total_weights = sum(var_list)
mean_portfolio_return = quicksum(var_list[i]*mean_return[i] for i in range(1158))
#var_list[0]*mean_return[0] + var_list[1]*mean_return[1] + var_list[2]*mean_return[2]
m.addConstr(total_weights, GRB.EQUAL, required_total_weights)
m.addConstr(PortVar, GRB.LESS_EQUAL, risk)
m.update()

m.setObjective(mean_portfolio_return, GRB.MAXIMIZE)
m.update()
m.optimize()


PortSd = math.sqrt(m.objval)
print('the portfolio standar deviation is: ',PortSd)

Optimize a model with 1 rows, 1158 columns and 1158 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [8e-09, 4e-01]
  Objective range  [8e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [5e-02, 5e-02]
Presolve time: 0.20s
Presolved: 121 rows, 1278 columns, 132059 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.259e+03
 Factor NZ  : 7.381e+03 (roughly 1 MByte of memory)
 Factor Ops : 5.979e+05 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.19313589e+00 -0.00000000e+00  1.16e+02 1.74e-01  1.50e-02     0s
   1   1.25625436e-02  1.60385588e-01  2.47e-01 1.37e-02  1.62e-04     0s
   2   1.22849790e-02  8.01985793e-02  2.71e-07 5.50e-03  5.84e-05     0s
   3   3.2560


# Multiple model and graph

In [13]:
risks=list(np.arange(0,0.051,0.005))[1:]
max_return=[]
for risk in risks:
    required_total_weights = 1.0
    var_list=[m.addVar(vtype=GRB.CONTINUOUS,name="Stock"+str(i)) for i in range(1158)]
    m.update()
    PortVar=quicksum(var_list[i]*covariances[i][j]*var_list[j] for i in range(1158) for j in range(1158))
    #PortVar = np.dot(var_list, np.dot(covariances,np.transpose(var_list)))

    # add constraints 
    total_weights = sum(var_list)
    mean_portfolio_return = quicksum(var_list[i]*mean_return[i] for i in range(1158))
    #var_list[0]*mean_return[0] + var_list[1]*mean_return[1] + var_list[2]*mean_return[2]
    m.addConstr(total_weights, GRB.EQUAL, required_total_weights)
    m.addConstr(PortVar, GRB.LESS_EQUAL, risk)
    m.update()

    m.setObjective(mean_portfolio_return, GRB.MAXIMIZE)
    m.update()
    m.optimize()
    max_return.append(m.objVal)
max_return   


Optimize a model with 2 rows, 2316 columns and 2316 nonzeros
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [8e-09, 4e-01]
  Objective range  [8e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [5e-03, 5e-02]
Presolve time: 0.22s
Presolved: 242 rows, 2556 columns, 264118 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.452e+04
 Factor NZ  : 1.476e+04 (roughly 1 MByte of memory)
 Factor Ops : 1.196e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.19313589e+00 -0.00000000e+00  1.16e+02 1.76e-01  1.57e-02     0s
   1   2.36111447e-02  3.25810312e-01  1.46e+00 1.36e-02  3.29e-04     0s
   2   1.12739274e-02  1.55276109e-01  1.61e-06 5.06e-04  5.67e-05     0s
   3   1.45

   3   1.67811223e-02  2.98332230e-01  1.04e-03 3.87e-04  3.71e-05     1s
   4   2.50466206e-02  1.05471659e-01  1.15e-09 9.41e-05  1.06e-05     1s
   5   3.35241044e-02  8.83967155e-02  6.15e-10 6.70e-05  7.21e-06     1s
   6   4.04059903e-02  8.39086586e-02  4.25e-10 5.72e-05  5.72e-06     1s
   7   4.68679109e-02  7.15047536e-02  2.39e-10 3.01e-05  3.24e-06     1s
   8   5.44152454e-02  5.89482015e-02  5.39e-14 6.06e-06  5.96e-07     1s
   9   5.46408450e-02  5.50371727e-02  2.44e-15 4.93e-07  5.21e-08     1s
  10   5.46572535e-02  5.46975165e-02  1.18e-13 4.01e-08  5.28e-09     1s
  11   5.46589534e-02  5.46604438e-02  2.39e-12 1.52e-09  1.96e-10     1s
  12   5.46589732e-02  5.46590131e-02  5.72e-11 5.15e-10  5.20e-12     1s

Barrier solved model in 12 iterations and 1.15 seconds
Optimal objective 5.46589732e-02

Optimize a model with 7 rows, 8106 columns and 8106 nonzeros
Model has 7 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range  

Optimal objective 5.84992885e-02

Optimize a model with 11 rows, 12738 columns and 12738 nonzeros
Model has 11 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [8e-09, 4e-01]
  Objective range  [8e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [5e-03, 5e-02]
Presolve time: 1.05s
Presolved: 1331 rows, 14058 columns, 1452649 nonzeros
Presolved model has 11 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.985e+04
 Factor NZ  : 8.119e+04 (roughly 7 MBytes of memory)
 Factor Ops : 6.576e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.19313589e+00 -0.00000000e+00  1.16e+02 1.78e-01  1.62e-02     2s
   1   2.32915703e-02  1.80968933e+00  1.30e+00 1.42e-02  3.18e-04     2s
   2   1.19153753e-02  8.07795926e-01  1.3

[0.04256872036807711,
 0.048271218144109576,
 0.05134956272686798,
 0.05324518690821791,
 0.05465897319501808,
 0.05581047133850665,
 0.05680622898371778,
 0.05769478234046987,
 0.05849928850947305,
 0.05922783696828898]

In [16]:
risks

[0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05]

In [20]:
for i in range(len(risks)):
    sql="insert into portfolio(expReturn,expRisk) values ({},{})"
    cur.execute(sql.format(round(max_return[i],5),risks[i]))
    db.commit()

In [21]:
var_list

[<gurobi.Var Stock0 (value 1.1697020046856051e-10)>,
 <gurobi.Var Stock1 (value 1.4495323374509178e-10)>,
 <gurobi.Var Stock2 (value 2.305549884121138e-10)>,
 <gurobi.Var Stock3 (value 7.954336120439523e-11)>,
 <gurobi.Var Stock4 (value 1.2450008481967463e-10)>,
 <gurobi.Var Stock5 (value 1.262740683772544e-10)>,
 <gurobi.Var Stock6 (value 8.601282832683385e-11)>,
 <gurobi.Var Stock7 (value 1.167775906652872e-10)>,
 <gurobi.Var Stock8 (value 1.0667534311662197e-10)>,
 <gurobi.Var Stock9 (value 1.6586329103750871e-10)>,
 <gurobi.Var Stock10 (value 1.9306165425482626e-10)>,
 <gurobi.Var Stock11 (value 1.4691823548242356e-10)>,
 <gurobi.Var Stock12 (value 8.756897028489186e-11)>,
 <gurobi.Var Stock13 (value 1.0283506796367343e-10)>,
 <gurobi.Var Stock14 (value 9.475969957987631e-11)>,
 <gurobi.Var Stock15 (value 2.3009353162001696e-09)>,
 <gurobi.Var Stock16 (value 8.063365623356267e-11)>,
 <gurobi.Var Stock17 (value 1.7223314241206431e-10)>,
 <gurobi.Var Stock18 (value 5.582310959703681e

In [22]:
a

[(0.0133757369187091,),
 (0.00325440556102433,),
 (0.00277063068283102,),
 (0.00301481545541941,),
 (0.00203377925592976,),
 (-0.00123944436463289,),
 (0.00364852878164484,),
 (0.00319011054883507,),
 (0.00436927380892203,),
 (0.00111003209618003,),
 (0.0041664069848769,),
 (0.00352827003910389,),
 (0.00476366937012172,),
 (0.00209899460851086,),
 (0.00432760767255483,),
 (0.00206919717317918,),
 (0.0022989792490959,),
 (0.00136983326570511,),
 (0.00394286888207393,),
 (0.00357697890748457,),
 (0.00400550707642252,),
 (0.00558544342870748,),
 (0.00804240321079234,),
 (-0.000503020585904367,),
 (0.00302164055605035,),
 (0.00183453461305198,),
 (0.00443960764006527,),
 (0.00177428115450318,),
 (0.00231675043126453,),
 (0.00441477394269316,),
 (-0.000356800642299451,),
 (0.00185938402758228,),
 (-0.000348738604765488,),
 (0.000201986207308738,),
 (2.34763883837941e-05,),
 (0.00115762618023055,),
 (0.00379427962735891,),
 (0.00195786364805589,),
 (0.00330736405044318,),
 (0.003729693342248

In [25]:
import pandas as pd 
pd.DataFrame(covariances)

,0,1,2,3,4,5,6,7,8,9,...,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157
0,0.013376,0.003254,0.002771,0.003015,0.002034,-0.001239,0.003649,0.003190,0.004369,0.001110,...,0.003532,0.002542,0.003865,0.004400,0.004782,0.000834,0.004078,0.008329,0.008298,0.004020
1,0.003254,0.023868,0.004237,0.001578,0.002114,0.002250,0.003812,0.006375,0.000304,0.000825,...,0.009827,0.002796,0.005158,0.007150,0.003692,-0.002061,0.003755,0.006252,0.008997,0.008563
2,0.002771,0.004237,0.019898,0.002158,0.002370,0.005486,0.003223,0.005139,0.001446,0.001253,...,0.005938,0.001726,0.002125,0.005625,0.006924,0.001486,0.002846,0.008673,0.007389,0.002822
3,0.003015,0.001578,0.002158,0.011670,0.003913,0.004294,0.003478,0.002404,0.003627,0.002430,...,0.002387,0.001425,0.001605,0.002347,0.005925,0.004429,0.005210,0.005371,0.002036,-0.000493
4,0.002034,0.002114,0.002370,0.003913,0.015009,0.003271,0.003639,0.000500,0.002372,0.001904,...,0.004761,0.000795,0.003248,0.006494,0.001719,0.004115,0.001212,-0.000174,0.002945,-0.000287
5,-0.001239,0.002250,0.005486,0.004294,0.003271,0.039089,0.002368,0.008492,0.004282,0.001506,...,0.003580,-0.000092,0.003217,0.011945,0.009001,0.002543,0.005274,-0.001499,-0.000730,0.002007
6,0.003649,0.003812,0.003223,0.003478,0.003639,0.002368,0.012566,0.004543,0.004398,0.002525,...,0.003567,0.001097,0.003317,0.005873,0.009517,0.003517,0.003235,0.006012,0.006441,0.003763
7,0.003190,0.006375,0.005139,0.002404,0.000500,0.008492,0.004543,0.043243,0.003298,0.003945,...,0.005900,0.002050,0.007276,0.003838,0.029032,-0.000529,0.004292,0.009099,0.015754,0.006423
8,0.004369,0.000304,0.001446,0.003627,0.002372,0.004282,0.004398,0.003298,0.020609,0.001622,...,0.002142,0.001825,0.003154,0.009068,0.007136,0.003139,0.003212,0.003296,0.007887,-0.001802
9,0.001110,0.000825,0.001253,0.002430,0.001904,0.001506,0.002525,0.003945,0.001622,0.003846,...,0.001441,0.000604,0.001532,0.001404,0.004297,0.000811,0.001676,0.004000,0.000937,0.000262


In [26]:
[2]+[3]

[2, 3]